# Proyecto de Consultoría – Finanzas (IIND 3400)
## Grupo 1 – Portafolio de Inversión Diversificado (Semestre 2025-20)
**Universidad de los Andes – Departamento de Ingeniería Industrial**  
**Profesor:** Sergio Cabrales  
**Asistente:** Gabriela Correa  

---

### **1. Introducción**
El presente proyecto tiene como objetivo estructurar un **portafolio de inversión diversificado** compuesto por **30 acciones listadas en la Bolsa de Valores de Nueva York (NYSE y NASDAQ)**.  
El trabajo está enmarcado en el curso de **Finanzas Corporativas**, y busca aplicar los conceptos de **riesgo, retorno, valuación, y optimización de portafolio** dentro de un contexto real de mercado.

De acuerdo con la asignación de sectores del curso (**Documento “FINANZAS 202520 - Asignación Sectores y Grupos Definitivos”**), al **Grupo 1** se le asignaron los siguientes **cuatro sectores económicos**:

1. **Tecnología – Tecnología de Consumo**  
2. **Energía – Energía Tradicional**  
3. **Salud – Farmacéutica**  
4. **Consumo – Ropa y Calzado**

El objetivo será analizar individualmente las acciones seleccionadas para cada sector (retornos, volatilidad, betas, valuación por dividendos y ratios financieros) y luego construir un portafolio optimizado que minimice el riesgo para un nivel de retorno determinado, siguiendo la teoría moderna de portafolio de Markowitz.

---

### **2. Composición del portafolio (30 acciones)**
Cada sector participa en proporciones ponderadas de acuerdo con su relevancia dentro del mercado y su contribución al equilibrio riesgo-retorno del portafolio total.  
Se seleccionaron **30 compañías** reconocidas internacionalmente, de alta liquidez y representativas de cada subsector.

#### **Tecnología – Tecnología de Consumo (9 acciones)**
| Empresa | Ticker | Bolsa | Justificación |
|:--------|:--------|:------|:---------------|
| Apple Inc. | AAPL | NASDAQ | Líder mundial en hardware y ecosistema de dispositivos personales. |
| Sony Group Corp. | SONY | NYSE | Diversificación entre electrónica de consumo, entretenimiento y gaming. |
| Alphabet Inc. (Google) | GOOGL | NASDAQ | Exposición a innovación digital y monetización publicitaria. |
| Microsoft Corp. | MSFT | NASDAQ | Productividad, servicios en la nube y presencia defensiva. |
| Amazon.com Inc. | AMZN | NASDAQ | E-commerce y ecosistema de consumo tecnológico. |
| Panasonic Holdings Corp. | PCRFY | NYSE | Electrónica y soluciones energéticas. |
| GoPro Inc. | GPRO | NASDAQ | Innovación en productos de nicho de consumo tecnológico. |
| Garmin Ltd. | GRMN | NYSE | Tecnología aplicada a wearables y navegación personal. |
| HP Inc. | HPQ | NYSE | Líder tradicional de hardware y generación de rentabilidad por dividendos. |

#### **Energía – Energía Tradicional (8 acciones)**
| Empresa | Ticker | Bolsa | Justificación |
|:--------|:--------|:------|:---------------|
| ExxonMobil Corp. | XOM | NYSE | Petrolera integrada con flujo de caja sólido. |
| Chevron Corp. | CVX | NYSE | Alta rentabilidad y presencia geográfica diversificada. |
| ConocoPhillips | COP | NYSE | Productor upstream eficiente y con expansión reciente. |
| BP p.l.c. | BP | NYSE | Presencia global y transición gradual hacia energía limpia. |
| Shell p.l.c. | SHEL | NYSE | Operaciones integradas globales y fuerte flujo operativo. |
| Valero Energy Corp. | VLO | NYSE | Refinería importante en EE.UU., diversificación industrial. |
| Marathon Petroleum Corp. | MPC | NYSE | Activo downstream con fuerte rendimiento. |
| Halliburton Co. | HAL | NYSE | Representa la rama de servicios petroleros dentro del sector. |

#### **Salud – Farmacéutica (8 acciones)**
| Empresa | Ticker | Bolsa | Justificación |
|:--------|:--------|:------|:---------------|
| Johnson & Johnson | JNJ | NYSE | Conglomerado farmacéutico defensivo y estable. |
| Pfizer Inc. | PFE | NYSE | Amplia presencia internacional y flujo de efectivo sólido. |
| Merck & Co., Inc. | MRK | NYSE | Innovación en oncología e inmunología. |
| Bristol Myers Squibb | BMY | NYSE | Portafolio diversificado en terapias innovadoras. |
| AbbVie Inc. | ABBV | NYSE | Fuerte flujo vía productos inmunológicos (Humira). |
| Eli Lilly & Co. | LLY | NYSE | Enfocada en biotecnología y alta valorización bursátil. |
| Amgen Inc. | AMGN | NASDAQ | Referente en terapias biotecnológicas consolidadas. |
| Gilead Sciences | GILD | NASDAQ | Especialización en antivirales y diversificación en biotecnología. |

#### **Consumo – Ropa y Calzado (5 acciones)**
| Empresa | Ticker | Bolsa | Justificación |
|:--------|:--------|:------|:---------------|
| Nike Inc. | NKE | NYSE | Líder global con fuerte presencia internacional. |
| Adidas AG (ADR) | ADDYY | NYSE | Competidor directo de Nike y comparativo europeo clave. |
| Lululemon Athletica | LULU | NASDAQ | Empresa de rápido crecimiento y márgenes premium. |
| VF Corporation | VFC | NYSE | Portfolio defensivo de marcas textiles clásicas. |
| Under Armour Inc. | UAA | NYSE | Segmento de consumo discrecional con potencial de rebote. |

---

### **3. Justificación de la estructura**
El portafolio sigue una distribución ponderada por exposición a capitalización y características de riesgo:

| Sector | % participación | Nº acciones |
|:-------|:----------------|:-------------|
| Tecnología | 30% | 9 |
| Energía | 25% | 8 |
| Salud | 25% | 8 |
| Consumo | 20% | 5 |

Esta asignación busca capturar crecimiento potencial en sectores cíclicos (tecnología, consumo) y estabilidad en sectores defensivos (energía, salud), logrando un equilibrio entre rendimiento esperado y volatilidad total.

---

**Próximos pasos:**  
En las siguientes secciones del notebook se realizará el análisis individual por activo (precios, retornos, ratios y betas), la construcción de matrices de covarianza y la optimización para determinar el portafolio de mínima varianza y frontera eficiente.